<a href="https://colab.research.google.com/github/evaaaaho1999/Text-Mining/blob/main/LDA2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## LDA & Word2vec

In [ ]:
b=np.load('/content/gdrive/MyDrive/Statistical computing Final project/files/tokens_all.npy',allow_pickle=True)
b=b.tolist()
b_train=b[0:41157]
c=list(b_train)

In [ ]:
#透過gensim以text_data建立字典
from gensim import corpora
dictionary =corpora.Dictionary(b_train)
#語料庫
corpus = [dictionary.doc2bow(token) for token in b_train]

In [ ]:
import gensim
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.word2vec import Word2Vec
num_topics = 150
#A multicore approach to decrease training time
LDAmodel = LdaMulticore(corpus=corpus,
              id2word=dictionary,
              num_topics=num_topics,
              workers=4,
              chunksize=2000,
              passes=7,
              alpha='asymmetric')

In [ ]:
def document_to_lda_features(lda_model, document):
    """ Transforms a bag of words document to features.
    It returns the proportion of how much each topic was
    present in the document.
    """
    topic_importances = LDAmodel.get_document_topics(document, minimum_probability=0)
    topic_importances = np.array(topic_importances)
    return topic_importances[:,1]

train['lda_features'] = list(map(lambda doc:
                                      document_to_lda_features(LDAmodel, doc),
                                      corpus))

In [ ]:
sentences = []
for sentence_group in b:
    sentences.extend(sentence_group)

print("Number of sentences: {}.".format(len(sentences)))
print("Number of texts: {}.".format(len(train)))

Number of sentences: 818236.
Number of texts: 41157.


In [ ]:
# Set values for various parameters
num_features = 200    # Word vector dimensionality
min_word_count = 3    # Minimum word count
num_workers = 4       # Number of threads to run in parallel
context = 6           # Context window size
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model
W2Vmodel = Word2Vec(sentences=sentences,
                    sg=1,
                    hs=0,
                    workers=num_workers,
                    size=num_features,
                    min_count=min_word_count,
                    window=context,
                    sample=downsampling,
                    negative=5,
                    iter=6)

In [ ]:
train2=train['lemma_text'].dropna().to_frame().reset_index()
train2=train2.drop(columns=['index'])
b_train2 = b_train[:]
b_train2.remove(b_train[33122])
b_train2.remove(b_train[33123])
b_train2.remove(b_train[33124])
b_train2.remove(b_train[39206])
b_train2.remove(b_train[39207])
b_train2.remove(b_train[39208])
c=list(b_train2)
len(c)

41151

In [ ]:
type(train['lemma_text'][1].split())

list

In [ ]:
def get_w2v_features(w2v_model, sentence_group):
    """ Transform a sentence_group (containing multiple lists
    of words) into a feature vector. It averages out all the
    word vectors of the sentence_group.
    """
    words = sentence_group
    # words = np.concatenate([np.expand_dims(i,axis=0) for i in sentence_group])# words in text
    index2word_set = set(w2v_model.wv.vocab.keys())  # words known to model
    
    featureVec = np.zeros(w2v_model.vector_size, dtype="float32")
    
    # Initialize a counter for number of words in a review
    nwords = 0
    # Loop over each word in the comment and, if it is in the model's vocabulary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            featureVec = np.add(featureVec, w2v_model[word])
            nwords += 1.

    # Divide the result by the number of words to get the average
    if nwords > 0:
        featureVec = np.divide(featureVec, nwords)
    return featureVec

train['w2v_features'] = list(map(lambda sen_group:
                                      get_w2v_features(W2Vmodel, sen_group),
                                      train['lemma_text']))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [ ]:
train.to_csv(r'/content/gdrive/MyDrive/Statistical computing Final project/files/train_with_feature.csv', index = False, header=True)

In [ ]:
train2=pd.read_csv("/content/gdrive/MyDrive/Statistical computing Final project/files/train_with_feature.csv",encoding='latin1')
train2

In [ ]:
# since train_data['lda_features'] and train_data['w2v_features'] don't have the needed shape and type yet,
# we first have to transform every entry
X_train_lda = np.array(list(map(np.array, train2.lda_features)))
X_train_w2v = np.array(list(map(np.array, train2.w2v_features)))

In [ ]:
y_train=train2['sentiment'].dropna().astype(str)
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
labelencoder.fit(y_train)
# labelencoder.fit(train['sentiment'])
y_train = labelencoder.transform(y_train)
len(y_train)

41151

In [ ]:
X_train_lda=np.delete(X_train_lda, 33122, 0)
X_train_lda=np.delete(X_train_lda, 33123, 0)
X_train_lda=np.delete(X_train_lda, 33124, 0)
X_train_lda=np.delete(X_train_lda, 39206, 0)
X_train_lda=np.delete(X_train_lda, 39207, 0)
X_train_lda=np.delete(X_train_lda, 39208, 0)
X_train_w2v=np.delete(X_train_w2v, 33122, 0)
X_train_w2v=np.delete(X_train_w2v, 33123, 0)
X_train_w2v=np.delete(X_train_w2v, 33124, 0)
X_train_w2v=np.delete(X_train_w2v, 39206, 0)
X_train_w2v=np.delete(X_train_w2v, 39207, 0)
X_train_w2v=np.delete(X_train_w2v, 39208, 0)
X_train_combined = np.append(X_train_lda, X_train_w2v)

In [ ]:
def get_cross_validated_model(model, param_grid, X, y, nr_folds=5):
    """ Trains a model by doing a grid search combined with cross validation.
    args:
        model: your model
        param_grid: dict of parameter values for the grid search
    returns:
        Model trained on entire dataset with hyperparameters chosen from best results in the grid search.
    """
    # train the model (since the evaluation is based on the logloss, we'll use neg_log_loss here)
    grid_cv = GridSearchCV(model, param_grid=param_grid, scoring='neg_log_loss', cv=nr_folds, n_jobs=-1, verbose=True)
    best_model = grid_cv.fit(X, y)
    # show top models with parameter values
    result_df = pd.DataFrame(best_model.cv_results_)
    show_columns = ['mean_test_score', 'mean_train_score', 'rank_test_score']
    for col in result_df.columns:
        if col.startswith('param_'):
            show_columns.append(col)
    display(result_df[show_columns].sort_values(by='rank_test_score').head())
    return best_model

In [ ]:
# store all models in a dictionary
models = dict()

In [ ]:
from sklearn.linear_model import LogisticRegression
clf=LogisticRegression(fit_intercept=True,C=1e15)
clf.fit(aaa,y_train)
print(clf.intercept_,clf.coef_)
#與第一種方法相比
print(weights)

In [ ]:
lr = LogisticRegression()
param_grid = {'penalty': ['none', 'l2']}
grid_cv =GridSearchCV(lr, param_grid=param_grid, scoring='neg_log_loss', cv=5, n_jobs=-1, verbose=True)
best_model = grid_cv.fit(X_train_lda, y_train)

In [ ]:
# LDA features only
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
lr = LogisticRegression()
param_grid = {'penalty': ['none', 'l2']}

best_lr_lda = get_cross_validated_model(lr, param_grid, X_train_lda, y_train)

models['best_lr_lda'] = best_lr_lda